In [2]:
import heapq
from collections import deque

romania_map = {
    "Arad": [("Zerind", 75), ("Sibiu", 140), ("Timisoara", 118)],
    "Zerind": [("Arad", 75), ("Oradea", 71)],
    "Oradea": [("Zerind", 71), ("Sibiu", 151)],
    "Sibiu": [("Arad", 140), ("Oradea", 151), ("Fagaras", 99), ("Rimnicu Vilcea", 80)],
    "Fagaras": [("Sibiu", 99), ("Bucharest", 211)],
    "Rimnicu Vilcea": [("Sibiu", 80), ("Pitesti", 97), ("Craiova", 146)],
    "Timisoara": [("Arad", 118), ("Lugoj", 111)],
    "Lugoj": [("Timisoara", 111), ("Mehadia", 70)],
    "Mehadia": [("Lugoj", 70), ("Drobeta", 75)],
    "Drobeta": [("Mehadia", 75), ("Craiova", 120)],
    "Craiova": [("Drobeta", 120), ("Rimnicu Vilcea", 146), ("Pitesti", 138)],
    "Pitesti": [("Rimnicu Vilcea", 97), ("Craiova", 138), ("Bucharest", 101)],
    "Bucharest": [("Fagaras", 211), ("Pitesti", 101), ("Giurgiu", 90), ("Urziceni", 85)],
    "Giurgiu": [("Bucharest", 90)],
    "Urziceni": [("Bucharest", 85), ("Hirsova", 98), ("Vaslui", 142)],
    "Hirsova": [("Urziceni", 98), ("Eforie", 86)],
    "Eforie": [("Hirsova", 86)],
    "Vaslui": [("Urziceni", 142), ("Iasi", 92)],
    "Iasi": [("Vaslui", 92), ("Neamt", 87)],
    "Neamt": [("Iasi", 87)]
}

heuristics = {
    "Arad": 366, "Bucharest": 0, "Craiova": 160, "Drobeta": 242, "Eforie": 161,
    "Fagaras": 176, "Giurgiu": 77, "Hirsova": 151, "Iasi": 226, "Lugoj": 244,
    "Mehadia": 241, "Neamt": 234, "Oradea": 380, "Pitesti": 100, "Rimnicu Vilcea": 193,
    "Sibiu": 253, "Timisoara": 329, "Urziceni": 80, "Vaslui": 199, "Zerind": 374
}


def reconstruct_path(parent_map, start, goal):
    path = []
    total_cost = 0
    current = goal

    while current != start:
        path.append(current)
        (p, c) = parent_map[current]
        total_cost += c
        current = p
    path.append(start)
    path.reverse()
    return path, total_cost

#breadth-First Search
def breadth_first_search(start, goal):
    if start == goal:
        return [start], 0

    visited = set()
    queue = deque()
    queue.append(start)
    parent_map = {}

    visited.add(start)

    while queue:
        current = queue.popleft()
        if current == goal:
            return reconstruct_path(parent_map, start, goal)

        for (neighbor, dist) in romania_map[current]:
            if neighbor not in visited:
                visited.add(neighbor)
                parent_map[neighbor] = (current, dist)
                queue.append(neighbor)

    return None, 0

#uniform cost search
def uniform_cost_search(start, goal):
    if start == goal:
        return [start], 0

    frontier = []
    heapq.heappush(frontier, (0, start))
    visited_costs = {start: 0}
    parent_map = {}

    while frontier:
        (cost_so_far, current) = heapq.heappop(frontier)

        if current == goal:
            return reconstruct_path(parent_map, start, goal)

        for (neighbor, dist) in romania_map[current]:
            new_cost = cost_so_far + dist
            if neighbor not in visited_costs or new_cost < visited_costs[neighbor]:
                visited_costs[neighbor] = new_cost
                parent_map[neighbor] = (current, dist)
                heapq.heappush(frontier, (new_cost, neighbor))

    return None, 0


#greedy best-first search
def greedy_best_first_search(start, goal):
    if start == goal:
        return [start], 0

    frontier = []
    heapq.heappush(frontier, (heuristics[start], start))
    visited = set()
    parent_map = {}

    while frontier:
        (_, current) = heapq.heappop(frontier)
        visited.add(current)

        if current == goal:
            return reconstruct_path(parent_map, start, goal)

        for (neighbor, dist) in romania_map[current]:
            if neighbor not in visited:
                visited.add(neighbor)
                parent_map[neighbor] = (current, dist)
                heapq.heappush(frontier, (heuristics[neighbor], neighbor))

    return None, 0


#iterative deepening depth-first search
def depth_limited_dfs(current, goal, limit, parent_map, visited, start):
    if current == goal:
        return True
    if limit == 0:
        return False

    visited.add(current)

    for (neighbor, dist) in romania_map[current]:
        if neighbor not in visited:
            parent_map[neighbor] = (current, dist)
            if depth_limited_dfs(neighbor, goal, limit - 1, parent_map, visited, start):
                return True
    return False

def iterative_deepening_dfs(start, goal):
    if start == goal:
        return [start], 0

    depth = 0
    while True:
        parent_map = {}
        visited = set()
        if depth_limited_dfs(start, goal, depth, parent_map, visited, start):
            return reconstruct_path(parent_map, start, goal)
        depth += 1


def main():
    print("Romania Map Search")
    print("Available cities:")
    for city in sorted(romania_map.keys()):
        print(city, end=", ")
    print("\n")

    start = input("Enter the start city: ").strip()
    goal = input("Enter the goal city: ").strip()

    if start not in romania_map or goal not in romania_map:
        print("Invalid city name(s). Please run again.")
        return

    # BFS
    bfs_path, bfs_cost = breadth_first_search(start, goal)

    # UCS
    ucs_path, ucs_cost = uniform_cost_search(start, goal)

    # Greedy
    greedy_path, greedy_cost = greedy_best_first_search(start, goal)

    # IDDFS
    iddfs_path, iddfs_cost = iterative_deepening_dfs(start, goal)

    print("\n1) Breadth-First Search")
    print("Path:", bfs_path)
    print("Path Cost:", bfs_cost)

    print("\n2) Uniform-Cost Search")
    print("Path:", ucs_path)
    print("Path Cost:", ucs_cost)

    print("\n3) Greedy Best-First Search")
    print("Path:", greedy_path)
    print("Path Cost:", greedy_cost)

    print("\n4) Iterative Deepening DFS")
    print("Path:", iddfs_path)
    print("Path Cost:", iddfs_cost)

    print("\n--- COMPARISON ---")
    paths_and_costs = [
        ("BFS", bfs_path, bfs_cost),
        ("UCS", ucs_path, ucs_cost),
        ("Greedy", greedy_path, greedy_cost),
        ("ID-DFS", iddfs_path, iddfs_cost)
    ]

    sorted_by_cost = sorted(paths_and_costs, key=lambda x: x[2])

    print("Paths in ascending order of cost:")
    for method, path, cost in sorted_by_cost:
        print(f"{method}: cost = {cost}, path = {path}")


main()


Romania Map Search
Available cities:
Arad, Bucharest, Craiova, Drobeta, Eforie, Fagaras, Giurgiu, Hirsova, Iasi, Lugoj, Mehadia, Neamt, Oradea, Pitesti, Rimnicu Vilcea, Sibiu, Timisoara, Urziceni, Vaslui, Zerind, 

Enter the start city: Hirsova
Enter the goal city: Arad

1) Breadth-First Search
Path: ['Hirsova', 'Urziceni', 'Bucharest', 'Fagaras', 'Sibiu', 'Arad']
Path Cost: 633

2) Uniform-Cost Search
Path: ['Hirsova', 'Urziceni', 'Bucharest', 'Pitesti', 'Rimnicu Vilcea', 'Sibiu', 'Arad']
Path Cost: 601

3) Greedy Best-First Search
Path: ['Hirsova', 'Urziceni', 'Bucharest', 'Fagaras', 'Sibiu', 'Arad']
Path Cost: 633

4) Iterative Deepening DFS
Path: ['Hirsova', 'Urziceni', 'Bucharest', 'Fagaras', 'Sibiu', 'Arad']
Path Cost: 633

--- COMPARISON ---
Paths in ascending order of cost:
UCS: cost = 601, path = ['Hirsova', 'Urziceni', 'Bucharest', 'Pitesti', 'Rimnicu Vilcea', 'Sibiu', 'Arad']
BFS: cost = 633, path = ['Hirsova', 'Urziceni', 'Bucharest', 'Fagaras', 'Sibiu', 'Arad']
Greedy: cos